# Z2-Higgs Model Dynamics

Reproducing Fig. 17 from [arXiv:2409.03747](https://arxiv.org/abs/2409.03747).
### Hamiltonian in (1 + 1)D:

The $Z_2$-Higgs model Hamiltonian is given by:

$$
\hat{H}_{Z_2} = -g \sum_{i=1}^{L-1} \hat{X}_{i,i+1} + U \sum_{i=1}^{L} \hat{n}_i^2 
- J \sum_{i=1}^{L-1} \left( \hat{a}^\dagger_i \hat{Z}_{i,i+1} \hat{a}_{i+1} + \text{h.c.} \right)
$$

We will simulate the Trotterized dynamics for $U = 0$.

---

### Trotterized Evolution:

We evolve initial states using a first-order Trotterization:

$$
e^{-i\hat{H}_{Z_2}t} \approx \left( \hat{U}_1 \hat{U}_2 \right)^r
$$

where $\Delta t = t/r$ and:

1. **Electric field on links:**
   $$
   \hat{U}_1 = \prod_{i} e^{-ig\hat{X}_{i,i+1}\Delta t}
   $$

2. **Hopping term for bosons between sites:**
   $$
   \hat{U}_2 = \prod_{i=1}^{L-1} e^{-iJ\hat{Z}_{i,i+1}(\hat{a}^\dagger_i \hat{a}_{i+1} + \hat{a}_i \hat{a}^\dagger_{i+1})\Delta t}
   $$

---

### Steps (to be removed before pushing to GitHub):

1. Work out how to compile Trotter steps into native architecture gates.
2. Simulate dynamics with the compiled circuits.
3. Determine the measurement scheme (e.g., do we need to use transmons to read out Fock numbers?).
4. Reproduce the plot (match the dynamics and decoherence effects as presented in Fig. 17).


In [2]:
import sys
import os

# Adjust the path based on your directory structure
module_path = os.path.abspath(os.path.join('..', '..'))  # Moves two directories up
if module_path not in sys.path:
    sys.path.append(module_path)
    
# Now you can import c2qa and other modules from bosonic-qiskit
import c2qa
import qiskit
import numpy as np
import c2qa.util as util
import matplotlib.pyplot as plt
import matplotlib

We consider a $L=5$-site chain prepared in the state $|\psi_0\rangle = |00100\rangle$. The limitation to five sites is not restrictive for short timescales, as the interactions are short-range, and excitations spread with a finite velocity. As long as the “lightcone” does not reach the boundary, the results obtained are the same as in a larger system. The qubits representing the $\mathbb{Z}_2$-Higgs model gauge fields must be initialized according to Gauss’s law, ensuring consistency between the qubits and the parity in each mode. Specifically, with the leftmost gauge field qubit in $|+\rangle$ and the sites in the Fock state $|00100\rangle$, the gauge field qubits on the links between the bosonic sites are initialized in $|++--\rangle$.


In [11]:
# Parameters
numberofmodes = 5
numberofqubits = numberofmodes - 1
numberofqubitspermode = 3
cutoff = 2 ** numberofqubitspermode

# Create registers
qmr = c2qa.QumodeRegister(num_qumodes=numberofmodes, num_qubits_per_qumode=numberofqubitspermode)  # Qumodes
qbr = qiskit.QuantumRegister(size=numberofqubits)  # Qubits for gauge fields
cbr = qiskit.ClassicalRegister(size=1)  # Classical register for readout
circuit = c2qa.CVCircuit(qmr, qbr, cbr)  # Circuit

# Initialize bosonic modes (Fock state |00100⟩)
fock_states = [0, 0, 1, 0, 0]  # Corresponds to |00100⟩
for i, fock_amplitude in enumerate(fock_states):
    coeffs = [0] * cutoff  # Initialize all coefficients to 0
    coeffs[fock_amplitude] = 1  # Set the amplitude for the desired Fock state
    circuit.cv_initialize(coeffs, qmr[i])  # Initialize each qumode

# Initialize qubits (|++--⟩)
plus_state = [1 / np.sqrt(2), 1 / np.sqrt(2)]  # Coefficients for |+⟩
minus_state = [1 / np.sqrt(2), -1 / np.sqrt(2)]  # Coefficients for |−⟩

# Apply initialization to each qubit
for i in range(numberofqubits):
    if i < 2:  # First two qubits to |+⟩
        circuit.initialize(plus_state, qbr[i])
    else:  # Last two qubits to |−⟩
        circuit.initialize(minus_state, qbr[i])


# Draw the circuit for visualization
# circuit.draw('mpl')

# state0, _, _ = c2qa.util.simulate(circuit)
# print(state0)

Now we need to define the gate sequence correspoding to a single Trotter step.